In [1]:
### load package
import pandas as pd
pd.set_option('display.max_rows', 500)
import numpy as np
import gurobipy as gp
import time
model_sanitized_path = 'C:/Users/52427/Documents/Cases/Case11_Baixiang/Baixiang_supply_chain_optimization/model_sanitized/'
from datetime import date

today = date.today()
today = str(today).replace('-','')

In [2]:
file_name = model_sanitized_path + 'model_input_20220616_EN_v2.xlsx'
xls = pd.ExcelFile(file_name)

customers_df = pd.read_excel(xls, 'Input_CustomerList')

factories_df = pd.read_excel(xls, 'Input_FactoryList')

products_df = pd.read_excel(xls, 'Input_ProductList')

demand_df = pd.read_excel(xls, 'Input_CustomerDemand')

trans_policies_df = pd.read_excel(xls, 'Input_TransportationPolicy')

prod_policies_df = pd.read_excel(xls, 'Input_CapacityByLineByProduct')

prod_line_policies_df = pd.read_excel(xls, 'Input_CapacityByLine')

factory_var_cost_df = pd.read_excel(xls, 'Input_FactoryVariableCost')

factory_fixed_cost_df = pd.read_excel(xls, 'Input_FactoryFixedCost')

inv_df = pd.read_excel(xls, 'Input_InventoryPolicy')

### load baseline data tables
hist_outbound_trans_df = pd.read_excel(xls, sheet_name='Input_HistShipmentFactoryToCust')

hist_inbound_trans_df = pd.read_excel(xls, sheet_name='Input_HistShipmentAmongFactory')

hist_prod_df = pd.read_excel(xls, sheet_name='Input_HistProduction')

hist_inv_df = pd.read_excel(xls, sheet_name='Input_HistInventoryLevel')

xls.close()

In [3]:
### create lists of basic model elements
customers = [x for x in customers_df['CustomerName']]

factories = [x for x in factories_df['FactoryName']]

products = [x for x in products_df['ProductName']]

months = range(1, 13)


### create dict of CN and EN
customers_dict = dict(zip(customers, ['CZ_'+ str(i) for i in range(1,len(customers)+1)]))

factories_dict = dict(zip(factories, ['FAC_'+ str(i) for i in range(1,len(factories)+1)]))

products_dict = dict(zip(products, ['PROD_'+ str(i) for i in range(1,len(products)+1)]))

destinations_dict = {**customers_dict, **factories_dict}

In [4]:
### replace CN names with EN names
customers_df.replace({'CustomerName': customers_dict}, inplace=True)

factories_df.replace({'FactoryName': factories_dict}, inplace=True)

products_df.replace({'ProductName': products_dict}, inplace=True)

demand_df.replace({'CustomerName': customers_dict, 'ProductName': products_dict}, inplace=True)

trans_policies_df.replace({'Origin': factories_dict, 'Destination': destinations_dict}, inplace=True)

prod_policies_df.replace({'FactoryName': factories_dict, 'ProductName': products_dict}, inplace=True)

prod_line_policies_df.replace({'FactoryName': factories_dict}, inplace=True)

factory_var_cost_df.replace({'FactoryName': factories_dict, 'ProductName': products_dict}, inplace=True)
### remove products that are not in product list
factory_var_cost_df = factory_var_cost_df[factory_var_cost_df['ProductName'].isin(list(products_dict.values()))]

factory_fixed_cost_df.replace({'FactoryName': factories_dict}, inplace=True)

inv_df.replace({'FactoryName': factories_dict}, inplace=True)

### load baseline data tables
hist_outbound_trans_df.replace({'Origin': factories_dict, 'Destination': customers_dict, 'ProductName': products_dict}, inplace=True)

hist_inbound_trans_df.replace({'Origin': factories_dict, 'Destination': factories_dict, 'ProductName': products_dict}, inplace=True)

hist_prod_df.replace({'FactoryName': factories_dict, 'ProductName': products_dict}, inplace=True)

hist_inv_df.replace({'FactoryName': factories_dict}, inplace=True)

In [5]:
### save model output template
writer = pd.ExcelWriter(model_sanitized_path + 'model_input_'+ today +'_EN_v2.xlsx', engine='xlsxwriter')

### input data tables
products_df.to_excel(writer, sheet_name='Input_ProductList', index=False)
customers_df.to_excel(writer, sheet_name='Input_CustomerList', index=False)
factories_df.to_excel(writer, sheet_name='Input_FactoryList', index=False)
factory_var_cost_df.to_excel(writer, sheet_name='Input_FactoryVariableCost', index=False)
factory_fixed_cost_df.to_excel(writer, sheet_name='Input_FactoryFixedCost', index=False)
trans_policies_df.to_excel(writer, sheet_name='Input_TransportationPolicy', index=False)
prod_policies_df.to_excel(writer, sheet_name='Input_CapacityByLineByProduct', index=False)
prod_line_policies_df.to_excel(writer, sheet_name='Input_CapacityByLine', index=False)
inv_df.to_excel(writer, sheet_name='Input_InventoryPolicy', index=False)
demand_df.to_excel(writer, sheet_name='Input_CustomerDemand', index=False)
hist_outbound_trans_df.to_excel(writer, sheet_name='Input_HistShipmentFactoryToCust', index=False)
hist_inbound_trans_df.to_excel(writer, sheet_name='Input_HistShipmentAmongFactory', index=False)
hist_prod_df.to_excel(writer, sheet_name='Input_HistProduction', index=False)
hist_inv_df.to_excel(writer, sheet_name='Input_HistInventoryLevel', index=False)

writer.save()

In [6]:
writer.close()

c:\Users\52427\.conda\envs\gurobienv\lib\site-packages\xlsxwriter\workbook.py:339: UserWarning: Calling close() on already closed file.
  warn("Calling close() on already closed file.")
